In [ ]:
import os
import numpy as np
%matplotlib inline
import numpy as np
import matplotlib.patches as mpatches
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pylab as plt
import matplotlib.patches as mpatches
import json
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


In [ ]:
emb_dir = "/home/pneekhara/SpeakerEmbeddings"
embeddings = {}
filenames = {}
for subdir in os.listdir(emb_dir):
    subdir_path = os.path.join(emb_dir, subdir)
    for file in os.listdir(subdir_path):
        if file.endswith(".npy"):
            with open(os.path.join(subdir_path, file), "rb") as f:
                embedding = np.load(f)
                #subdir_name, fname = file.split("-")
                #subdir_name = subdir_name + " " + fname.split("_")[0]
                if subdir in embeddings:
                    embeddings[subdir].append( embedding )
                    filenames[subdir].append( file )
                else:
                    embeddings[subdir] = [ embedding ]
                    filenames[subdir] = [ file ]
print (len(embeddings['9017']))

In [ ]:
def mscatter(x,y, ax=None, m=None, **kw):
    import matplotlib.markers as mmarkers
    #ax = ax or plt.gca()
    sc = ax.scatter(x,y,**kw)
    if (m is not None) and (len(m)==len(x)):
        paths = []
        for marker in m:
            if isinstance(marker, mmarkers.MarkerStyle):
                marker_obj = marker
            else:
                marker_obj = mmarkers.MarkerStyle(marker)
            path = marker_obj.get_path().transformed(
                        marker_obj.get_transform())
            paths.append(path)
        sc.set_paths(paths)
    return sc

def visualize_embeddings(embedding_dict_np, filenames, title = "TSNE", max_num_samples_per_spk = 1000):
    # extract num_speakers 10. With new speaker key will have num_speakers+1 speakers in the TSNE plot
    # keys = []
    label_list =[]
    color =[]
    marker_shape = []
    color_idx = 0
    universal_embed_list=[]
    handle_list=[]
    _unique_speakers = {}
    
    marker_list = ['<', '*', 'h', 'X', 's', 'H', 'D', 'd', 'P', 'v', '^', '>', '8', 'p']
    kidx = 0
    
    filenames_labelled = []
    for key in embedding_dict_np:
        speaker = key
        
        _num_samples = len(embedding_dict_np[key])
        if max_num_samples_per_spk is not None:
            _num_samples = min(len(embedding_dict_np[key]), max_num_samples_per_spk)
            
        
        universal_embed_list += embedding_dict_np[key][:_num_samples]
        filenames_labelled += [ (fname, None) for fname in filenames[key][:_num_samples] ]
        
        print('key:', key, 'length', len(embedding_dict_np[key]))
        
        #id_color = plt.cm.tab20(_unique_speakers[speaker])
        id_color = plt.cm.tab20(kidx)
        color_element = [id_color] * _num_samples
        color += color_element
        _marker_shape = [ "o" ] * _num_samples
        marker_shape += _marker_shape
        _label = key
        handle_list.append(mpatches.Patch(color = id_color, label=_label))
        
        kidx += 1
        
   
    print ("fitting tsne")
    speaker_embeddings = TSNE(n_components=2, random_state=0).fit_transform(universal_embed_list)
    speaker_embeddings = StandardScaler().fit_transform(speaker_embeddings)
    print (speaker_embeddings.shape)
#     db = SpectralClustering(assign_labels='discretize', n_clusters=2,random_state=0).fit(speaker_embeddings)
    db = AgglomerativeClustering(n_clusters=2, linkage='single').fit(speaker_embeddings)
    labels = db.labels_
    filenames_labelled = [ (filenames_labelled[lidx][0], labels[lidx]) for lidx in range(len(labels))]
    #print (len(labels), labels)
    color = [plt.cm.tab20(label) for label in labels]
    
    plt.rcParams["figure.figsize"] = (12, 12)
    #fig = plt.figure(figsize=(12, 12))
    #fig.add_subplot(121)
    #ax = plt.axes()
    #ax.set_facecolor("gainsboro")
    #plt.scatter(speaker_embeddings[:, 0], speaker_embeddings[:, 1], marker = marker_shape,  c=color, s=30)
    mscatter(speaker_embeddings[:, 0], speaker_embeddings[:, 1], plt, m = marker_shape,  c=color, s=10)
   
    plt.legend(handles=handle_list,title="Chapter ID")
    plt.title(title)
    
    return filenames_labelled

In [ ]:
filenames_labelled = visualize_embeddings( { '9017' : embeddings['9017'] } , filenames, max_num_samples_per_spk = None)

In [ ]:
def filename_to_wav_path(filename):
    base_path = "/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup/wav/9017_clean"
    chapter_id, wavname = filename.split("-")
    wavname = wavname.replace(".npy", ".wav")
    wavdir = os.path.join(base_path, chapter_id)
    wavpath = os.path.join(wavdir, wavname)
    return wavpath
    

In [ ]:
grouped_files = {0 : [], 1: []}
for fname_label in filenames_labelled:
    grouped_files[fname_label[1]].append(fname_label[0])

In [ ]:
import IPython.display as ipd

for label in [0,1]:
    print ("**************")
    print ("Cluster {}".format(label))
    for fidx in range(10):
        wav_path = filename_to_wav_path(grouped_files[label][fidx])
        ipd.display(ipd.Audio(wav_path, rate=44100))

In [ ]:
for key in grouped_files:
    print (key, len(grouped_files[key]))